In [1]:
import os
import sys
import gc
import copy
import ast
import cv2
from PIL import Image, ImageDraw
import time
import math
from collections import defaultdict, Counter
from pathlib import Path
import pickle
import random
import argparse
import warnings
from typing import Any

import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
import cv2
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import pydicom
import nibabel as nib
from scipy.ndimage import zoom

In [2]:
# from pathlib import Path
# import pandas as pd

# # 行番号でディレクトリを分岐する例
# # 行番号でディレクトリを分岐する例
# def choose_root(idx: int) -> Path:
#     return '/content/png'

# # ソート用キー
# def png_index(path: Path) -> int:
#     stem = path.stem
#     if '_' in stem:
#         try:
#             return int(stem.rsplit('_', 1)[1])
#         except ValueError:
#             pass
#     return -1

# # 3) 行単位で fill する関数
# def fill_sorted_files(row):

#     sid = row['SeriesInstanceUID']
#     # ここを row.name ではなく row['fold'] や row['SomeCol'] に変えてもOK
#     idx = row.name
#     save_root = choose_root(idx)
#     series_dir = Path(save_root) / sid
#      # 既にリストが入っているならそのまま
#     if isinstance(row['sorted_files'], list):
#         return [x.replace('/kaggle/input/rsna-intracranial-aneurysm-detection/series', save_root).replace('.dcm', '.png') for x in row['sorted_files']]

#     pngs = sorted(series_dir.glob('*.png'), key=png_index)
#     return [str(p) for p in pngs]

# def get_mask_slices(mask_path):
#     """NIfTIマスクを読み込み、ラベルがあるスライスのインデックスを返す"""
#     mask = nib.load(mask_path).get_fdata()
#     return [i for i in range(mask.shape[2]) if np.any(mask[..., i] > 0)]

In [3]:
series_to_remove = ['1.2.826.0.1.3680043.8.498.75712554178574230484227682423862727306', # 読み込みエラー
                    '1.2.826.0.1.3680043.8.498.82768897201281605198635077495114055892', # 読み込みエラー
                    '1.2.826.0.1.3680043.8.498.10063454172499468887877935052136698373', # z_spacingエラー
                    '1.2.826.0.1.3680043.8.498.22157965342587174310173115980837533982', # 読み込みエラー
                   ]

df = pd.read_csv('/kaggle/input/rsna2025-extra/train_add_metadata_v3.csv')
df_loc = pd.read_csv('/kaggle/input/rsna-intracranial-aneurysm-detection/train_localizers.csv')
df['sorted_files'] = df['sorted_files'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)
df['PixelSpacing'] = df['PixelSpacing'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)

df = df[~df['SeriesInstanceUID'].isin(series_to_remove)].reset_index(drop=True)
series_root = Path('/kaggle/input/rsna-intracranial-aneurysm-detection/segmentations')
series_names = list(series_root.glob('*'))
series_names = [series_name.stem for series_name in series_names]
df_seg = df[df['SeriesInstanceUID'].isin(series_names)].reset_index(drop=True)
df_seg['reverse'] = False
df_seg['file_name'] = df_seg['SeriesInstanceUID'].apply(lambda x: series_root  / (x + '.nii'))
df_seg['file_mask_name'] =  df_seg['SeriesInstanceUID'].apply(lambda x: series_root / (x + '_cowseg.nii'))
df_seg

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,SeriesInstanceUID,PatientAge,PatientSex,Modality,Left Infraclinoid Internal Carotid Artery,Right Infraclinoid Internal Carotid Artery,Left Supraclinoid Internal Carotid Artery,Right Supraclinoid Internal Carotid Artery,Left Middle Cerebral Artery,Right Middle Cerebral Artery,...,TransferSyntaxUID,SeriesPath,OrientationLabel,AnteriorUp,OrientationLabelEncoded,ModalityEncoded,fold,reverse,file_name,file_mask_name
0,1.2.826.0.1.3680043.8.498.10035643165968342618...,62,Female,CTA,0,0,0,1,1,0,...,NaN,/kaggle/input/rsna-intracranial-aneurysm-detec...,AXIAL,True,0,1,4,False,/kaggle/input/rsna-intracranial-aneurysm-detec...,/kaggle/input/rsna-intracranial-aneurysm-detec...
1,1.2.826.0.1.3680043.8.498.10076056930521523789...,40,Female,MRA,0,0,0,1,0,0,...,NaN,/kaggle/input/rsna-intracranial-aneurysm-detec...,AXIAL,True,0,0,4,False,/kaggle/input/rsna-intracranial-aneurysm-detec...,/kaggle/input/rsna-intracranial-aneurysm-detec...
2,1.2.826.0.1.3680043.8.498.10188636688783982623...,68,Female,MRA,1,0,0,0,0,0,...,NaN,/kaggle/input/rsna-intracranial-aneurysm-detec...,AXIAL,True,0,0,4,False,/kaggle/input/rsna-intracranial-aneurysm-detec...,/kaggle/input/rsna-intracranial-aneurysm-detec...
3,1.2.826.0.1.3680043.8.498.10410600166004340343...,58,Female,MRA,0,0,0,0,0,1,...,NaN,/kaggle/input/rsna-intracranial-aneurysm-detec...,AXIAL,True,0,0,3,False,/kaggle/input/rsna-intracranial-aneurysm-detec...,/kaggle/input/rsna-intracranial-aneurysm-detec...
4,1.2.826.0.1.3680043.8.498.10540586847553109495...,49,Female,MRI T2,0,0,0,0,0,1,...,NaN,/kaggle/input/rsna-intracranial-aneurysm-detec...,AXIAL,True,0,2,3,False,/kaggle/input/rsna-intracranial-aneurysm-detec...,/kaggle/input/rsna-intracranial-aneurysm-detec...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,1.2.826.0.1.3680043.8.498.97793793786310123662...,72,Male,MRI T2,0,0,0,0,0,0,...,NaN,/kaggle/input/rsna-intracranial-aneurysm-detec...,CORONAL,False,1,2,0,False,/kaggle/input/rsna-intracranial-aneurysm-detec...,/kaggle/input/rsna-intracranial-aneurysm-detec...
173,1.2.826.0.1.3680043.8.498.97954169904067078117...,40,Male,MRA,0,0,0,0,0,0,...,NaN,/kaggle/input/rsna-intracranial-aneurysm-detec...,AXIAL,True,0,0,1,False,/kaggle/input/rsna-intracranial-aneurysm-detec...,/kaggle/input/rsna-intracranial-aneurysm-detec...
174,1.2.826.0.1.3680043.8.498.98123758735027035609...,47,Male,CTA,0,0,1,0,0,0,...,NaN,/kaggle/input/rsna-intracranial-aneurysm-detec...,CORONAL,False,1,1,3,False,/kaggle/input/rsna-intracranial-aneurysm-detec...,/kaggle/input/rsna-intracranial-aneurysm-detec...
175,1.2.826.0.1.3680043.8.498.98133633346919790888...,71,Male,MRI T1post,0,0,0,1,0,0,...,NaN,/kaggle/input/rsna-intracranial-aneurysm-detec...,AXIAL,True,0,3,3,False,/kaggle/input/rsna-intracranial-aneurysm-detec...,/kaggle/input/rsna-intracranial-aneurysm-detec...


In [4]:
df_seg['Modality'].unique()

array(['CTA', 'MRA', 'MRI T2', 'MRI T1post'], dtype=object)

In [5]:
def ensure_dir(p: str | Path):
    Path(p).mkdir(parents=True, exist_ok=True)

def robust_window(img3d: np.ndarray, low=1, high=99) -> tuple[float, float]:
    finite = np.isfinite(img3d)
    if finite.any():
        vmin = float(np.percentile(img3d[finite], low))
        vmax = float(np.percentile(img3d[finite], high))
    else:
        vmin = float(np.min(img3d))
        vmax = float(np.max(img3d))
    if vmin == vmax:
        vmax = vmin + 1.0
    return vmin, vmax

def pct_normalize(img, p_min, p_max):
    vmin = np.percentile(img, p_min)
    vmax = np.percentile(img, p_max)
    img = np.clip(img, vmin, vmax)
    img = (img - vmin) / (vmax - vmin + 1e-6)
    img = (img*255).astype(np.uint8)
    return img

def value_normalize(img, vmin, vmax):
    img = np.clip(img, vmin, vmax)
    img = (img - vmin) / (vmax - vmin + 1e-6)
    img = (img*255).astype(np.uint8)
    return img
    
def resample_yxz(arr: np.ndarray,
                 spacing: tuple[float, float, float],
                 new_spacing: tuple[float, float, float],
                 order: int) -> np.ndarray:
    """
    (Y,X,Z) を spacing -> new_spacing へリサンプリング。
    order=1: 画像（線形）, order=0: マスク（最近傍）
    """
    sy, sx, sz = spacing
    nsy, nsx, nsz = new_spacing
    zy, zx, zz = sy / nsy, sx / nsx, sz / nsz
    mode = "nearest" if order == 0 else "constant"
    return zoom(arr, zoom=(zy, zx, zz), order=order, mode=mode)

def extract_bbox_2d(mask2d: np.ndarray):
    """2Dバイナリから (xmin, ymin, xmax, ymax) を返す。なければ None"""
    coords = np.argwhere(mask2d > 0)
    if coords.size == 0:
        return None
    rows, cols = coords[:, 0], coords[:, 1]
    ymin, ymax = int(rows.min()), int(rows.max())
    xmin, xmax = int(cols.min()), int(cols.max())
    return (xmin, ymin, xmax, ymax)

def extract_mip_and_bbox(mask_yxz: np.ndarray, axis: int):
    """(Y,X,Z) マスクを axis に沿ってMIP。bbox を返す。axis: 0=Y,1=X,2=Z"""
    mip = np.max(mask_yxz > 0, axis=axis).astype(np.uint8)
    bbox = extract_bbox_2d(mip)
    return mip, bbox

def draw_bbox_on_uint8(img_uint8: np.ndarray, bbox, color=(255, 255, 0), width=2) -> Image.Image:
    """
    img_uint8: (H,W) グレースケール uint8
    bbox: (xmin, ymin, xmax, ymax)
    """
    rgb = np.stack([img_uint8]*3, axis=-1)
    pil = Image.fromarray(rgb)
    if bbox is not None:
        xmin, ymin, xmax, ymax = bbox
        drw = ImageDraw.Draw(pil)
        # PIL の矩形は inclusive に見えるので、枠が1pxずれるのを避けてそのまま描画
        for w in range(width):
            drw.rectangle([xmin - w, ymin - w, xmax + w, ymax + w], outline=color, width=1)
    return pil

def yolo_from_bbox(bbox, w: int, h: int, class_id: int = 0) -> str:
    """
    YOLO v5/v8 形式 (class cx cy w h), 座標は 0-1 正規化
    bbox: (xmin, ymin, xmax, ymax)
    """
    xmin, ymin, xmax, ymax = bbox
    bw = xmax - xmin + 1
    bh = ymax - ymin + 1
    cx = xmin + bw / 2
    cy = ymin + bh / 2
    return f"{class_id} {cx / w:.6f} {cy / h:.6f} {bw / w:.6f} {bh / h:.6f}"

# ------------------ 保存メイン関数 ------------------

def save_detection_samples(
    volume: np.ndarray,                     # (Y,X,Z)
    mask: np.ndarray | None,                # (Y,X,Z), 0/1 or labels
    spacing: tuple[float,float,float] | None,   # (sy,sx,sz)
    out_root: str | Path,
    case_id: str,
    modality: str,                          # 例: "CT","MRI"
    plane: str = "unknown",                 # 取得面のメタ情報: "axial"/"coronal"/"sagittal"/"unknown" 等
    axes: tuple[int,int,int] = (0,1,2),     # 保存する軸（central slice）
    new_spacing: tuple[float,float,float] | None = None,  # 等方化したい場合
    bbox_mode: str = "mip",                 # "mip" or "slice"
    class_id: int = 0,
    skip_if_no_bbox: bool = True,
    window: tuple[float,float] | None = None,  # (vmin,vmax) Noneならrobust
):
    assert volume.ndim == 3, "volume must be (Y,X,Z)"
    # --- resample (任意) ---
    if spacing is not None:
        if new_spacing is None:
            iso = float(min(spacing))
            new_spacing = (iso, iso, iso)
        vol_r = resample_yxz(volume, spacing, new_spacing, order=1).astype(np.uint8)
        msk_r = resample_yxz(mask,   spacing, new_spacing, order=0) if mask is not None else None
    else:
        vol_r, msk_r = volume, mask

    Y, X, Z = vol_r.shape
    cy, cx, cz = Y//2, X//2, Z//2

    
    # 軸→中央スライスと MIP 軸の対応
    # axis=0: fix Y -> slice shape (X,Z); MIP along axis=0
    # axis=1: fix X -> slice shape (Y,Z); MIP along axis=1
    # axis=2: fix Z -> slice shape (Y,X); MIP along axis=2
    for axis in axes:
        if axis == 0:
            img2d = vol_r[cy, :, :]                 # (X,Z)
            msk2d = (msk_r[cy, :, :] > 0).astype(np.uint8) if msk_r is not None else None
        elif axis == 1:
            img2d = vol_r[:, cx, :]                 # (Y,Z)
            msk2d = (msk_r[:, cx, :] > 0).astype(np.uint8) if msk_r is not None else None
        elif axis == 2:
            img2d = vol_r[:, :, cz]                 # (Y,X)
            msk2d = (msk_r[:, :, cz] > 0).astype(np.uint8) if msk_r is not None else None
        else:
            raise ValueError("axes must be subset of {0,1,2}")

        # BBox
        bbox = None
        if msk_r is not None:
            if bbox_mode == "slice":
                bbox = extract_bbox_2d(msk2d)
            elif bbox_mode == "mip":
                _, bbox = extract_mip_and_bbox(msk_r, axis=axis)
            else:
                raise ValueError("bbox_mode must be 'mip' or 'slice'")

        if bbox is None and skip_if_no_bbox:
            continue  # ラベルなしはスキップ（必要なら画像だけ保存に変更可）

        # 出力先: out_root/<modality>/<plane>/axis{0|1|2}/{images,labels_yolo,labels_voc,overlay}
        base = Path(out_root) / modality / plane / f"axis{axis}"
        p_img   = base / "images"
        p_yolo  = base / "labels_yolo"
        p_voc   = base / "labels_voc"
        p_ovl   = base / "overlay"
        for p in (p_img, p_yolo, p_voc, p_ovl): ensure_dir(p)

        # 画像保存（8bit）
        h, w = img2d.shape
        Image.fromarray(img2d, mode="L").save(p_img / f"{case_id}.png")

        # ラベル保存
        if bbox is not None:
            xmin, ymin, xmax, ymax = bbox
            (p_yolo / f"{case_id}.txt").write_text(
                yolo_from_bbox(bbox, w=w, h=h, class_id=class_id) + "\n", encoding="utf-8"
            )
            (p_voc / f"{case_id}.txt").write_text(
                f"{xmin} {ymin} {xmax} {ymax}\n", encoding="utf-8"
            )
        else:
            # 空ファイルで残したいならこちら（skip_if_no_bbox=Falseの時など）
            (p_yolo / f"{case_id}.txt").write_text("", encoding="utf-8")

        # overlay
        ovl = draw_bbox_on_uint8(img2d, bbox, color=(255,255,0), width=2)
        ovl.save(p_ovl / f"{case_id}.png")

    return True

In [6]:
df_seg['SeriesInstanceUID']

0      1.2.826.0.1.3680043.8.498.10035643165968342618...
1      1.2.826.0.1.3680043.8.498.10076056930521523789...
2      1.2.826.0.1.3680043.8.498.10188636688783982623...
3      1.2.826.0.1.3680043.8.498.10410600166004340343...
4      1.2.826.0.1.3680043.8.498.10540586847553109495...
                             ...                        
172    1.2.826.0.1.3680043.8.498.97793793786310123662...
173    1.2.826.0.1.3680043.8.498.97954169904067078117...
174    1.2.826.0.1.3680043.8.498.98123758735027035609...
175    1.2.826.0.1.3680043.8.498.98133633346919790888...
176    1.2.826.0.1.3680043.8.498.98665804676684243068...
Name: SeriesInstanceUID, Length: 177, dtype: object

In [ ]:
for i, row in tqdm(df_seg.iterrows(), total=len(df_seg)):
    sid = row['SeriesInstanceUID']
    modality = row['Modality']
    plane = row['OrientationLabel']
    series_path = row['file_name']
    series_mask_path = row['file_mask_name']
    y_spacing, x_spacing = row['PixelSpacing']
    z_spacing = row['z_spacing']
    imgs = nib.load(series_path).get_fdata()
    masks = nib.load(series_mask_path).get_fdata()
    masks = np.rot90(masks)
    
    imgs = np.rot90(imgs)
    if modality=='CTA':
        imgs = value_normalize(imgs, -100, 600)
    else:
        imgs = pct_normalize(imgs, 1, 99)
    ok = save_detection_samples(
        volume=imgs,
        mask=(masks>0).astype(np.uint8),
        spacing=(y_spacing, x_spacing, z_spacing),
        out_root="./detetion_annotation",
        case_id=sid,
        modality=modality,
        plane=plane,          # ★ 取得面のメタ情報（フォルダ分け用）
        axes=(0,1,2),           # 保存したい軸（例: (2,) だけ保存なども可）
        new_spacing=(1.0,1.0,1.0),
        bbox_mode="mip",        # or "slice"
        class_id=0,
        skip_if_no_bbox=True,                # None ならロバスト窓(1-99pct)で8bit化
    )

  6%|▌         | 11/177 [00:45<12:17,  4.44s/it]